In [1]:
# import modules
from datetime import datetime
from pandas_datareader import data as pdr
import yfinance as yf
import pandas as pd

In [2]:
df = pd.read_csv('stock_code.csv')
print(df.shape)
len(df['stock_code'].unique())

(1000, 2)


1000

In [3]:
df2 = df.to_dict('dict')
df2

{'stock_company': {0: '7-ELEVEN MALAYSIA HOLDINGS BERHAD',
  1: 'ABF MALAYSIA BOND INDEX FUND',
  2: 'ABLE GLOBAL BERHAD',
  3: 'ABLEGROUP BERHAD',
  4: 'ABM FUJIYA BERHAD',
  5: 'ACE INNOVATE ASIA BERHAD',
  6: 'ACME HOLDINGS BERHAD',
  7: 'ACO GROUP BERHAD',
  8: 'ADVANCE INFORMATION MARKETING BERHAD',
  9: 'ADVANCE SYNERGY BERHAD',
  10: 'ADVANCECON HOLDINGS BERHAD',
  11: 'ADVANCED PACKAGING TECHNOLOGY (M) BHD',
  12: 'ADVENTA BERHAD',
  13: 'AE MULTI HOLDINGS BERHAD',
  14: 'AEMULUS HOLDINGS BERHAD',
  15: 'AEON CO. (M) BHD',
  16: 'AEON CREDIT SERVICE (M) BERHAD',
  17: 'AFFIN BANK BERHAD',
  18: 'AGESON BERHAD',
  19: 'AHB HOLDINGS BERHAD',
  20: 'AHMAD ZAKI RESOURCES BERHAD',
  21: 'AIMFLEX BERHAD',
  22: 'AIRASIA X BERHAD',
  23: 'AJINOMOTO (MALAYSIA) BERHAD',
  24: 'AJIYA BERHAD',
  25: 'AL-`AQAR HEALTHCARE REIT',
  26: 'ALAM MARITIM RESOURCES BERHAD',
  27: 'ALCOM GROUP BERHAD',
  28: 'ALDRICH RESOURCES BERHAD',
  29: 'ALLIANCE BANK MALAYSIA BERHAD',
  30: 'ALLIANZ MALAYSIA 

# Connect to database

In [4]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote 
from datetime import timedelta

In [5]:
# connect to database
conn_string = 'postgresql://postgres:%s@localhost/dividend_investing'% quote('@SMWHot4')

#connect
db = create_engine(conn_string)
conn = db.connect()
conn = psycopg2.connect(conn_string)
conn.autocommit = True
cursor = conn.cursor()

In [6]:
# using sql query to get recent date
sql2 ='''SELECT DISTINCT TO_CHAR("stockDate",'YYYYMMDD') FROM stock_price
WHERE "stockDate" = (SELECT MAX("stockDate") FROM stock_price);'''
cursor.execute(sql2)
recent_date = cursor.fetchone()

conn.commit()
conn.close()

#20230117 or 20231223
year = int(recent_date[0][0:4])    
month = int(recent_date[0][4:6])
day =  int(recent_date[0][6:8])
print(recent_date,year,month,day)

('20230116',) 2023 1 16


In [8]:
start = datetime(year,month,day)+timedelta(days=1)
end = datetime.now()-timedelta(days=1)
print(end)

# put it into a csv file to track the latest date
lat_date = pd.DataFrame({"date":[start]})
lat_date.to_csv("latest_date.csv", mode='a')

2023-01-17 21:29:15.696667


In [9]:
for i in range(len(df2['stock_code'])):
    
    stock = yf.Ticker(df2['stock_code'][i]) 
    table = stock.history(start=start, end=end)
    
    frame=pd.DataFrame(table)
    frame['stock code'] = df2['stock_code'][i]
    frame['stock name']= df2['stock_company'][i]
    frame.to_csv('recent_stock.csv', mode='a', index=True, header=False)

Got error from yahoo api for ticker 9474.KL, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- 9474.KL: No timezone found, symbol may be delisted
Got error from yahoo api for ticker 8761.KL, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- 8761.KL: No timezone found, symbol may be delisted
Got error from yahoo api for ticker 7205.KL, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- 7205.KL: No timezone found, symbol may be delisted
Got error from yahoo api for ticker 2925.KL, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- 2925.KL: No timezone found, symbol may be delisted
0400GA.KL: Period '1mo' is invalid, must be one of ['1d', '5d']
0400GB.KL: Period '1mo' is invalid, must be one of ['1d', '5d']
5108.KL: No data found for this date range, symbol may be delisted
0401GA.KL: Period '1mo' is invalid, must be one of ['1d', '5d']
Got error

In [10]:
find = pd.read_csv('recent_stock.csv', header = None)

In [11]:
find

,0,1,2,3,4,5,6,7,8,9
0,2023-01-17 00:00:00+08:00,1.950,1.980,1.920,1.970,394800,0.0,0.0,5250.KL,7-ELEVEN MALAYSIA HOLDINGS BERHAD
1,2023-01-17 00:00:00+08:00,1.164,1.164,1.164,1.164,0,0.0,0.0,0800EA.KL,ABF MALAYSIA BOND INDEX FUND
2,2023-01-17 00:00:00+08:00,1.450,1.460,1.440,1.450,219600,0.0,0.0,7167.KL,ABLE GLOBAL BERHAD
3,2023-01-17 00:00:00+08:00,0.140,0.140,0.135,0.135,81500,0.0,0.0,7086.KL,ABLEGROUP BERHAD
4,2023-01-17 00:00:00+08:00,0.430,0.430,0.430,0.430,0,0.0,0.0,5198.KL,ABM FUJIYA BERHAD
...,...,...,...,...,...,...,...,...,...,...
983,2023-01-17 00:00:00+08:00,0.280,0.280,0.270,0.275,6891800,0.0,0.0,0250.KL,YX PRECIOUS METALS BHD
984,2023-01-17 00:00:00+08:00,0.385,0.385,0.385,0.385,0,0.0,0.0,7028.KL,ZECON BERHAD
985,2023-01-17 00:00:00+08:00,0.070,0.070,0.065,0.070,245500,0.0,0.0,2283.KL,ZELAN BERHAD
986,2023-01-17 00:00:00+08:00,0.035,0.040,0.035,0.040,5136000,0.0,0.0,0094.KL,ZEN TECH INTERNATIONAL BERHAD


In [12]:
find.rename(columns = {0: 'stockDate', 1: 'stockOpen', 2: 'stockHigh', 3: 'stockLow', 4: 'stockClose', 5: 'stockVolume', 6: 'stockDividend', 7: 'stockSplits', 8: 'stockCode', 9: 'stockCompanyName'}, inplace = True)

In [13]:
find

,stockDate,stockOpen,stockHigh,stockLow,stockClose,stockVolume,stockDividend,stockSplits,stockCode,stockCompanyName
0,2023-01-17 00:00:00+08:00,1.950,1.980,1.920,1.970,394800,0.0,0.0,5250.KL,7-ELEVEN MALAYSIA HOLDINGS BERHAD
1,2023-01-17 00:00:00+08:00,1.164,1.164,1.164,1.164,0,0.0,0.0,0800EA.KL,ABF MALAYSIA BOND INDEX FUND
2,2023-01-17 00:00:00+08:00,1.450,1.460,1.440,1.450,219600,0.0,0.0,7167.KL,ABLE GLOBAL BERHAD
3,2023-01-17 00:00:00+08:00,0.140,0.140,0.135,0.135,81500,0.0,0.0,7086.KL,ABLEGROUP BERHAD
4,2023-01-17 00:00:00+08:00,0.430,0.430,0.430,0.430,0,0.0,0.0,5198.KL,ABM FUJIYA BERHAD
...,...,...,...,...,...,...,...,...,...,...
983,2023-01-17 00:00:00+08:00,0.280,0.280,0.270,0.275,6891800,0.0,0.0,0250.KL,YX PRECIOUS METALS BHD
984,2023-01-17 00:00:00+08:00,0.385,0.385,0.385,0.385,0,0.0,0.0,7028.KL,ZECON BERHAD
985,2023-01-17 00:00:00+08:00,0.070,0.070,0.065,0.070,245500,0.0,0.0,2283.KL,ZELAN BERHAD
986,2023-01-17 00:00:00+08:00,0.035,0.040,0.035,0.040,5136000,0.0,0.0,0094.KL,ZEN TECH INTERNATIONAL BERHAD


In [14]:
find['stockCode']=find['stockCode'].str.slice(0, -3)

In [15]:
find

,stockDate,stockOpen,stockHigh,stockLow,stockClose,stockVolume,stockDividend,stockSplits,stockCode,stockCompanyName
0,2023-01-17 00:00:00+08:00,1.950,1.980,1.920,1.970,394800,0.0,0.0,5250,7-ELEVEN MALAYSIA HOLDINGS BERHAD
1,2023-01-17 00:00:00+08:00,1.164,1.164,1.164,1.164,0,0.0,0.0,0800EA,ABF MALAYSIA BOND INDEX FUND
2,2023-01-17 00:00:00+08:00,1.450,1.460,1.440,1.450,219600,0.0,0.0,7167,ABLE GLOBAL BERHAD
3,2023-01-17 00:00:00+08:00,0.140,0.140,0.135,0.135,81500,0.0,0.0,7086,ABLEGROUP BERHAD
4,2023-01-17 00:00:00+08:00,0.430,0.430,0.430,0.430,0,0.0,0.0,5198,ABM FUJIYA BERHAD
...,...,...,...,...,...,...,...,...,...,...
983,2023-01-17 00:00:00+08:00,0.280,0.280,0.270,0.275,6891800,0.0,0.0,0250,YX PRECIOUS METALS BHD
984,2023-01-17 00:00:00+08:00,0.385,0.385,0.385,0.385,0,0.0,0.0,7028,ZECON BERHAD
985,2023-01-17 00:00:00+08:00,0.070,0.070,0.065,0.070,245500,0.0,0.0,2283,ZELAN BERHAD
986,2023-01-17 00:00:00+08:00,0.035,0.040,0.035,0.040,5136000,0.0,0.0,0094,ZEN TECH INTERNATIONAL BERHAD


In [16]:
del find['stockDividend']

In [17]:
del find['stockSplits']

In [18]:
del find['stockVolume']

In [19]:
find

,stockDate,stockOpen,stockHigh,stockLow,stockClose,stockCode,stockCompanyName
0,2023-01-17 00:00:00+08:00,1.950,1.980,1.920,1.970,5250,7-ELEVEN MALAYSIA HOLDINGS BERHAD
1,2023-01-17 00:00:00+08:00,1.164,1.164,1.164,1.164,0800EA,ABF MALAYSIA BOND INDEX FUND
2,2023-01-17 00:00:00+08:00,1.450,1.460,1.440,1.450,7167,ABLE GLOBAL BERHAD
3,2023-01-17 00:00:00+08:00,0.140,0.140,0.135,0.135,7086,ABLEGROUP BERHAD
4,2023-01-17 00:00:00+08:00,0.430,0.430,0.430,0.430,5198,ABM FUJIYA BERHAD
...,...,...,...,...,...,...,...
983,2023-01-17 00:00:00+08:00,0.280,0.280,0.270,0.275,0250,YX PRECIOUS METALS BHD
984,2023-01-17 00:00:00+08:00,0.385,0.385,0.385,0.385,7028,ZECON BERHAD
985,2023-01-17 00:00:00+08:00,0.070,0.070,0.065,0.070,2283,ZELAN BERHAD
986,2023-01-17 00:00:00+08:00,0.035,0.040,0.035,0.040,0094,ZEN TECH INTERNATIONAL BERHAD


In [20]:
find['stockOpen']=find['stockOpen'].round(3)
find['stockHigh']=find['stockHigh'].round(3)
find['stockLow']=find['stockLow'].round(3)
find['stockClose']=find['stockClose'].round(3)

In [21]:
find[find['stockCode']=='0008']

,stockDate,stockOpen,stockHigh,stockLow,stockClose,stockCode,stockCompanyName
956,2023-01-17 00:00:00+08:00,0.38,0.38,0.38,0.38,0008,WILLOWGLEN MSC BERHAD


In [22]:
find

,stockDate,stockOpen,stockHigh,stockLow,stockClose,stockCode,stockCompanyName
0,2023-01-17 00:00:00+08:00,1.950,1.980,1.920,1.970,5250,7-ELEVEN MALAYSIA HOLDINGS BERHAD
1,2023-01-17 00:00:00+08:00,1.164,1.164,1.164,1.164,0800EA,ABF MALAYSIA BOND INDEX FUND
2,2023-01-17 00:00:00+08:00,1.450,1.460,1.440,1.450,7167,ABLE GLOBAL BERHAD
3,2023-01-17 00:00:00+08:00,0.140,0.140,0.135,0.135,7086,ABLEGROUP BERHAD
4,2023-01-17 00:00:00+08:00,0.430,0.430,0.430,0.430,5198,ABM FUJIYA BERHAD
...,...,...,...,...,...,...,...
983,2023-01-17 00:00:00+08:00,0.280,0.280,0.270,0.275,0250,YX PRECIOUS METALS BHD
984,2023-01-17 00:00:00+08:00,0.385,0.385,0.385,0.385,7028,ZECON BERHAD
985,2023-01-17 00:00:00+08:00,0.070,0.070,0.065,0.070,2283,ZELAN BERHAD
986,2023-01-17 00:00:00+08:00,0.035,0.040,0.035,0.040,0094,ZEN TECH INTERNATIONAL BERHAD


In [23]:
find.dtypes

stockDate            object
stockOpen           float64
stockHigh           float64
stockLow            float64
stockClose          float64
stockCode            object
stockCompanyName     object
dtype: object

In [24]:
find['stockDate'] = pd.to_datetime(find["stockDate"])

find['stockDate'] = pd.to_datetime(find["stockDate"].dt.strftime('%Y-%m-%d'))


In [25]:
find

,stockDate,stockOpen,stockHigh,stockLow,stockClose,stockCode,stockCompanyName
0,2023-01-17,1.950,1.980,1.920,1.970,5250,7-ELEVEN MALAYSIA HOLDINGS BERHAD
1,2023-01-17,1.164,1.164,1.164,1.164,0800EA,ABF MALAYSIA BOND INDEX FUND
2,2023-01-17,1.450,1.460,1.440,1.450,7167,ABLE GLOBAL BERHAD
3,2023-01-17,0.140,0.140,0.135,0.135,7086,ABLEGROUP BERHAD
4,2023-01-17,0.430,0.430,0.430,0.430,5198,ABM FUJIYA BERHAD
...,...,...,...,...,...,...,...
983,2023-01-17,0.280,0.280,0.270,0.275,0250,YX PRECIOUS METALS BHD
984,2023-01-17,0.385,0.385,0.385,0.385,7028,ZECON BERHAD
985,2023-01-17,0.070,0.070,0.065,0.070,2283,ZELAN BERHAD
986,2023-01-17,0.035,0.040,0.035,0.040,0094,ZEN TECH INTERNATIONAL BERHAD


In [26]:
find.dtypes

stockDate           datetime64[ns]
stockOpen                  float64
stockHigh                  float64
stockLow                   float64
stockClose                 float64
stockCode                   object
stockCompanyName            object
dtype: object

In [27]:
len(find['stockCode'].unique())

988

In [28]:
len(find['stockDate'].unique())

1

In [34]:
# connect to database
conn_string = 'postgresql://postgres:%s@localhost/dividend_investing'% quote('@SMWHot4')

#connect
db = create_engine(conn_string)
conn = db.connect()


find.to_sql("stock_price", con=conn, if_exists='append',
          index=False,index_label=False)
conn = psycopg2.connect(conn_string)
conn.autocommit = True
cursor = conn.cursor()

# sql1 = '''select * from stock_price;'''
# cursor.execute(sql1)
# count=0
# for i in cursor.fetchall():
#     count+=1
#     print(i)

conn.commit()
conn.close()